In [1]:
import re # to handle regular expression
import demoji  # for emojis handle
import random # to generate random number
import inflect  # to handle number to words
import numpy as np # scitific calculation 
import pandas as pd # data manipulation 
from bs4 import BeautifulSoup # handle html tag
import matplotlib.pyplot as plt # plot any display

import torch # deep learning handle
import torchtext # nlp handle
import torch.nn as nn # NN handle
import torch.optim as optim # optimizer handle
import torch.nn.functional as F # all type to DL funciton
from torch.nn.functional import one_hot # encoder-decoder
from torchtext.data.utils import get_tokenizer # tokenize 
print('Successfully import all the libraries')

Successfully import all the libraries


In [2]:
# Reading the text 
file_path = 'human_chat.txt'
with open(file_path,"r") as f:
    lines = f.readlines()

In [3]:
type(lines[1])

str

In [4]:
soup = BeautifulSoup(str(lines), 'html.parser')
text = soup.get_text()

In [6]:
# Preprocessing of the datasets

def preprocessing_text(text):
    # Remove HTML tag 
    soup = BeautifulSoup(str(text), 'html.parser')
    text = soup.get_text()

    p = inflect.engine() #101
    demoji.download_codes()

    # remove emojis
    text = demoji.replace(text,"")

    # Remove mentions of "Human 1 and Human 2"
    text = re.sub(r'\b(?:Human 1|Human 2)\b:?', " ", text)

    # Replace numbers with words 
    text = re.sub(r'\b\d+\b', lambda x:p.number_to_words(x.group()), text)

    # Remove special charaters, keeping only alphabetic and space
    text = re.sub('[^a-zA-Z\s]',' ',text)

    # Replace specific unicode spaces wht standard spaces and trim
    text = text.replace(u'\xa0', u' ').replace('\u200a', ' ').strip()

    return text



In [7]:
preprocesed_lines = [preprocessing_text(line) for line in lines]

/tmp/ipykernel_37217/607048165.py:9: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
/tmp/ipykernel_37217/607048165.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(text), 'html.parser')


In [8]:
preprocesed_lines[30:100]

['Hi',
 'Hey there  What s up',
 'Nothing much  how you doin',
 'I m in New York this week for Thanksgiving  I m squatting in the office today and I caught up with an old friend of mine  D',
 'Oh wow  Sounds like fun  When was the last time you had seen this friend',
 'The last time in New York  back in June',
 'Ohh okay  I was going to say if it had been a long time maybe it d be awkward',
 'Haha  I guess if it s been a very long time there s almost too many life events to catch up on   especially recently',
 'Oh really  Has a lot changed in your life recently',
 'Haha it s probably too much to go into at the moment  Let s just say life is an exciting experience  How about you',
 'Ahhh sounds exciting indeed  My life is pretty bland  I like routine  but sometimes I wish I had more time for adventures',
 'What kinds of adventures   Any ones that I would be able to join you on',
 'Hmmmm  I really want to try bull riding  Do you have any interest in that',
 'I d love to try  Can we sched

In [9]:
# Tokenize the words
tokenizer = get_tokenizer('basic_english')
tokenized_conv = [tokenizer(conv) for conv in preprocesed_lines]

In [10]:
tokenized_conv[2]

['one', 'where', 'i', 'get', 'to', 'meet', 'lots', 'of', 'different', 'people']

In [11]:
tokenized_conv[3]

['what',
 'was',
 'the',
 'most',
 'number',
 'of',
 'people',
 'you',
 'have',
 'ever',
 'met',
 'during',
 'a',
 'holiday']

In [12]:
tokenized_conv

[['hi'],
 ['what', 'is', 'your', 'favorite', 'holiday'],
 ['one',
  'where',
  'i',
  'get',
  'to',
  'meet',
  'lots',
  'of',
  'different',
  'people'],
 ['what',
  'was',
  'the',
  'most',
  'number',
  'of',
  'people',
  'you',
  'have',
  'ever',
  'met',
  'during',
  'a',
  'holiday'],
 ['hard', 'to', 'keep', 'a', 'count', 'maybe', 'twenty', 'five'],
 ['which', 'holiday', 'was', 'that'],
 ['i', 'think', 'it', 'was', 'australia'],
 ['do', 'you', 'still', 'talk', 'to', 'the', 'people', 'you', 'met'],
 ['not',
  'really',
  'the',
  'interactions',
  'are',
  'usually',
  'short',
  'lived',
  'but',
  'it',
  's',
  'fascinating',
  'to',
  'learn',
  'where',
  'people',
  'are',
  'coming',
  'from',
  'and',
  'what',
  'matters',
  'to',
  'them'],
 ['yea',
  'me',
  'too',
  'i',
  'feel',
  'like',
  'god',
  'often',
  'puts',
  'strangers',
  'in',
  'front',
  'of',
  'you',
  'and',
  'gives',
  'you',
  'an',
  'opportunity',
  'to',
  'connect',
  'with',
  'them',

## ```build_vocab_from_iterator```

```build_vocab_from_iterator``` function in the ```torchtext.vocab``` modeule is used to create vocabulary from an iterable of tokenized data. This vocabulary is essential for converting textual data into numerical form.

## Parameters:

### ```tokenized_conv(iterator)```:
This is the main data input to the function. It should be an iterator(like a ```generator``` or a ```list```) that yields sequences to tokens. Each sequences represents a document or an example in your dataset.

### ```min_freq(int,optional):```
This parameter specifics the mimimum frequency a token must have to be inclued in the vocabulary. Tokens that appear fewer than ```min_freq``` times are excluded form the vocabulary. This is useful for removing rare words which might be typos or irrelevant to most analysis.

### ```specials(list of str, optional)```:
This is a list of special tokens that you want to add to the vocabulary. Common special tokens include: A padding token used to equalize the lenghts of sequence.(or for unknown): A token used to represent out-of-vocabulary words during inference, or when a word appears that is not int the traning vocabualry. 

### ```special_first(bool, optional)```:
Determine the ordering of special tokens in the vocabulary. If ```True```, special tokens are added at the beginning of the vocabulary. This can be helpful for certain models where token indices are significant(e.g., models using embadding layers might have specific handling for lower indices.)

In [13]:
features_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    specials = ['<pad>', '<oov>'],
    special_first = True
)

In [17]:
target_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    min_freq=1
)

In [18]:
features_vocab_total_words = len(features_vocab)
target_vocab_toal_words = len(target_vocab)
print("Features Vocab Length:", features_vocab_total_words)
print("Features Vocab Length:", target_vocab_toal_words)

Features Vocab Length: 2749
Features Vocab Length: 2747


In [19]:
# making ngrams from the converstation
def make_ngrams(tokenized_text):
    list_ngrams = []
    for i in range(1, len(tokenized_text)):
        ngram_sequence = tokenized_text[:i+1]
        list_ngrams.append(ngram_sequence)
    return list_ngrams


In [20]:
ngram_list = []
for tokenized_con in tokenized_conv:
    ngram_list.extend(make_ngrams(tokenized_conv))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f68d70a99d0>>
Traceback (most recent call last):
  File "/home/mahfuz/anaconda3/envs/nlp/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
